<a href="https://colab.research.google.com/github/abooabdirahman/Search-Engine/blob/master/HamoyeRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pandas.util.testing as tm
df_test = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

column_names = {'T1':'Temperature in kitchen', 'RH_1': 'Humidity in kitchen', 
                'T2':  'Temperature in living-room', 'RH_2': 'Humidity in living-room', 'T3': 'Temperature in laundry',
                'RH_3': 'Humidity in laundry room', 'T4': 'Temperature in office room','RH_4': 'Humidity in office room',
                'T5': 'Temperature in bathroom', 'RH_5': 'Humidity in bathroom', 'T6': 'Temperature outside the building (north side)',
                'RH_6': 'Humidity outside the building (north side)', 'T7': 'Temperature in ironing room', 'RH_7': 'Humidity in ironing room',
                'T8': 'Temperature in teenager room 2', 'RH_8': 'Humidity in teenager room 2', 'T9': 'Temperature in parents room',
                'RH_9': 'Humidity in parents room', 'T_out': ' Temperature outside (from Chievres weather station)', 
                'Press_mm_hg': 'Pressure (from Chievres weather station)', 'RH_out': 'Humidity outside (from Chievres weather station)',
                'Windspeed': 'Wind speed (from Chievres weather station)', 'Visibility': 'Visibility (from Chievres weather station)',
                'Tdewpoint': 'Tdewpoint (from Chievres weather station)', 'rv1': 'Random variable 1', 'rv2': 'Random variable 2'}
df_test = df_test.rename(columns=column_names)
df_test.head()
                

,date,Appliances,lights,Temperature in kitchen,Humidity in kitchen,Temperature in living-room,Humidity in living-room,Temperature in laundry,Humidity in laundry room,Temperature in office room,Humidity in office room,Temperature in bathroom,Humidity in bathroom,Temperature outside the building (north side),Humidity outside the building (north side),Temperature in ironing room,Humidity in ironing room,Temperature in teenager room 2,Humidity in teenager room 2,Temperature in parents room,Humidity in parents room,Temperature outside (from Chievres weather station),Pressure (from Chievres weather station),Humidity outside (from Chievres weather station),Wind speed (from Chievres weather station),Visibility (from Chievres weather station),Tdewpoint (from Chievres weather station),Random variable 1,Random variable 2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [ ]:
df_new = df_test.drop(["date", "lights"], axis = 1)
df_new.head()

,Appliances,Temperature in kitchen,Humidity in kitchen,Temperature in living-room,Humidity in living-room,Temperature in laundry,Humidity in laundry room,Temperature in office room,Humidity in office room,Temperature in bathroom,Humidity in bathroom,Temperature outside the building (north side),Humidity outside the building (north side),Temperature in ironing room,Humidity in ironing room,Temperature in teenager room 2,Humidity in teenager room 2,Temperature in parents room,Humidity in parents room,Temperature outside (from Chievres weather station),Pressure (from Chievres weather station),Humidity outside (from Chievres weather station),Wind speed (from Chievres weather station),Visibility (from Chievres weather station),Tdewpoint (from Chievres weather station),Random variable 1,Random variable 2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_new)
df_scaler = pd.DataFrame(scaler.transform(df_new),columns=df_new.columns)

y = df_scaler['Appliances']
X = df_scaler.drop(['Appliances'], axis = 1)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)



In [36]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)

ridg_pred=ridge_reg.predict(X_test)
from sklearn.metrics import mean_squared_error
MAE = mean_absolute_error(y_test, ridg_pred)
round(MAE, 3)



0.05

In [49]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001, max_iter=10000, tol=0.00001)
lasso_reg.fit(X_train, y_train)
#lasso_trans = lasso_reg.transform(X_train, y_train)
lasso_pred = lasso_reg.predict(X_test)


In [28]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression(n_jobs=5)
lin_reg.fit(X_train,y_train)
lin_pred = lin_reg.predict(X_test)

lin_pred
#print(lin_reg.score(X_train,y_train))
#print(lin_reg.score(X_test,y_test))

array([0.03322207, 0.24411599, 0.03400024, ..., 0.06844707, 0.10032325,
       0.05722198])

In [30]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_test, lin_pred)
round(MAE, 3)

#lin_reg.score(X_test,y_test)

0.05

In [50]:
from sklearn.metrics import r2_score
r2_value = r2_score(y_test, lin_pred)
round(r2_value, 3)

#import numpy as np

from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
round(rmse, 3)

0.094

In [32]:
rs_value = np.sum(np.square(y_test - lin_pred))
round(rs_value, 3)


45.348

In [23]:
lin_reg.coef_

array([0.8910771])

In [47]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(lin_reg, X_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

final_weights['Lasso_weight' ]!=0

0     False
1     False
2     False
3     False
4      True
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18     True
19    False
20    False
21    False
22    False
23    False
24    False
25     True
Name: Lasso_weight, dtype: bool